In [ ]:
feature_engineering_for_split_dataset

In [ ]:
数据集天数位置：1-16

In [ ]:
数据集B的处理ipynb

In [ ]:
分离数据集提取特征
可预见这个ipynb又会很长

In [ ]:
数据集分别读入，不一次性读入

In [ ]:
常见的特征就是统计量：最大，最小，均值，☆中位数（下次可以添加，这次没考虑到），
和，峰度，偏度；计数特征，比率特征，二次融合特征等等

In [ ]:
思路：
1 先考虑之前的特征有没有问题——
比如：看过热度视频的个数——感觉比例比较好吧？因为看过多少也当做特征，有统计。
2 仔细考虑序列性特征的，最后的变化。不然只有均值统计，很容易特征根本无法区分最后弃坑的那种。

### train1-16

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
table123 = pd.read_csv('dataset_B_for_model_method/split_origin_dataset/table123_1_16.csv')

In [3]:
table123.head()

,user_id,register_day,register_type,device_type,app_launch_day,video_create_day,video_create_day_count
0,8,9.0,0.0,34.0,9.0,NaN,NaN
1,8,9.0,0.0,34.0,10.0,NaN,NaN
2,8,9.0,0.0,34.0,11.0,NaN,NaN
3,8,9.0,0.0,34.0,13.0,NaN,NaN
4,73,2.0,1.0,947.0,2.0,NaN,NaN


In [4]:
act = pd.read_csv('dataset_B_for_model_method/split_origin_dataset/act_1_16.csv')

In [5]:
act.head()

,user_id,activity_day,activity_page,video_id,author_id,action_type,activity_count
0,8,9,3,1955966,309219,0,2
1,8,9,0,1340816,12378,0,1
2,8,9,0,3488372,107008,0,1
3,8,9,0,1705402,37639,0,2
4,8,9,0,3710937,1312718,0,1


In [6]:
y = pd.read_csv('dataset_B_for_model_method/split_origin_dataset/y_1_16.csv')

In [7]:
y.head()

,user_id
0,8
1,1212424
2,1310730
3,819211
4,1376267


In [ ]:
【注】在这里尽力将之前写的代码变成def或者集成为大片的

#### 登录特征

In [ ]:
聚集list

In [8]:
def agg_list(table,column):
    launch_day_list = table.groupby('user_id')[column].agg(lambda x: list(x)).reset_index()
    launch_day_list.columns = ['user_id',column]
    
    # 增加一列间隔list
    launch_day_list['launch_gap_list'] = -1
    
    list_gap = []
    for item in launch_day_list['app_launch_day']:
        list_a = []
        if len(item) != 1:
            for i in range(1,len(item)):
                list_a.append(item[i] - item[i-1])
            list_gap.append(list_a)
        else:
            list_gap.append([-1])
    launch_day_list['launch_gap_list'] = list_gap
                          
    return launch_day_list

In [ ]:
提取天的特征

In [9]:
from scipy import stats

In [10]:
def day_feature_launch(table,time_gap_right):
    
    # 天数的特征
    list_max = [] # 最大的天数（最靠近预测日期的天数）
    list_count = [] # 登录天数总计
    list_average = [] # 平均登录天数
    for item in table['app_launch_day']:
        list_max.append(time_gap_right - item[-1]) # 取差而不是直接取最近的天数
        list_count.append(len(item))
        list_average.append ( len(item) / (item[-1] - item[0] + 1) )
        # print(item)    
    
    print('各种特征的shape')
    print(len(list_max),len(list_count),len(list_average))
    
    table['launch_day_near_pred'] = list_max
    table['launch_day_sum'] = list_count
    table['launch_day_sum_average'] = list_average
    
    # 间隔的特征
    gap_max = [] # 最大间隔
    gap_min = [] # 最小间隔
    gap_average = [] # 平均间隔
    gap_std = [] # 间隔方差
    gap_skew = [] # 间隔的偏度：衡量分布是左偏还是右偏，小于0左偏，说明过了最高值，下降的比上升快（更有可能弃坑）
    gap_kurtosis = [] # 间隔的峰度：大于0说明尖峰是陡升的，小于0则说明爬升较缓和
    for item in table['launch_gap_list']:
        if -1 not in item:            
            gap_max.append(np.max(item))
            gap_min.append(np.min(item))
            gap_average.append(np.mean(item))
            gap_std.append(np.std(item))
            gap_skew.append(stats.skew(item))
            gap_kurtosis.append(stats.kurtosis(item))
        else:
            gap_max.append(-1)
            gap_min.append(-1)
            gap_average.append(-1)
            gap_std.append(-1)
            # 考虑到取值范围，调整
            gap_skew.append(-99)
            gap_kurtosis.append(-99)
    
    table['launch_day_gap_max'] = gap_max
    table['launch_day_gap_ain'] = gap_min
    table['launch_day_gap_average'] = gap_average
    table['launch_day_gap_std'] = gap_std
    table['launch_day_gap_skew'] = gap_skew
    table['launch_day_gap_kurtosis'] = gap_kurtosis
    
    # 连续间隔特征
    gap_max_continue = [] # 全区间的最大连续间隔：注意1代表连续2天都有记录（注意1和2是差1的！），0代表没有连续的
    for item in table['launch_gap_list']:
        gap = -1 # =0代表没有1出现，即没有连续x天的记录。-1代表没有间隔
        if -1 not in item:
            gap = 0
            # 只有有1才可能gap不等于0
            if 1 in item:
                gap = 1 # 1的个数，至少1个
                left = 0
                right = 0
                switch = 0 # 用于控制间隔是否开始的开关
                # [2 3 1 1 2 3 1 1 1]
                for i in range(len(item)):           
                    # 开始计算间隔长度
                    if switch == 1 and item[i] == 1:
                        right = i
                    # 到1的新出发点
                    elif switch == 0 and item[i] == 1:
                        switch = 1
                        left = i
                        right = i
                    # 进入新的间隔停止期，遭遇非1，计算间隔长度
                    elif switch == 1 and item[i] != 1:
                        switch = 0
                        temp = right - left + 1
                        # 只记录最大的gap
                        if temp > gap:
                            gap = temp
                    # 进入非1的区间，不需要做任何事
                    # switch == 0 and item[i] != 1
                    else:
                        continue
                    
                     # 必须考虑是不是最后一个值了，此时对于是否是1，有不同的处理
                    if i == len(item) - 1:
                        # 此时需要计算一遍gap
                        if item[i] == 1:
                            temp = right - left + 1
                            # 只记录最大的gap
                            if temp > gap:
                                gap = temp
                                
        gap_max_continue.append(gap)       
        
    # 从后往前数的最长连续记录
    gap_max_right_continue = []
    for item in table['launch_gap_list']:
        gap = -1 # 没有间隔只能= -1
        if -1 not in item:
            gap = 0
            # 最后一个值是1才往前走
            if item[-1] == 1:
                gap = 1 # 1的个数
                # 从后往前的循环，从倒数第2个开始
                for i in range(len(item)-2,-1,-1):
                    if item[i] == 1:
                        gap = gap + 1
                    else:
                        break
        gap_max_right_continue.append(gap)
    
    # 比较两个gap是否数量相同，作为第3个特征，相同 = 0，不同 = 1（不区分左右，没有明显的意义）
    gap_continue_flag = []
    for i in range(len(gap_max_continue)):
        if gap_max_continue[i] == -1 or gap_max_right_continue[i] == -1:
            gap_continue_flag.append(-1) # 没有间隔
        elif gap_max_continue[i] == gap_max_right_continue[i]:
            gap_continue_flag.append(0)
        else:
            gap_continue_flag.append(1)
    
    table['launch_day_gap_max_continue'] = gap_max_continue
    table['launch_day_gap_max_right_continue'] = gap_max_right_continue
    table['launch_day_gap_max_continue_flag'] = gap_continue_flag
    
    
    # 从后往前数的，序列变得越来越稀疏的长度：即正向看的话，间隔值是否越来越大
    gap_change_large_right_continue = []
    for item in table['launch_gap_list']:
        gap = -1
        if -1 not in item:
            gap = 0
            for i in range(len(item)-1,0,-1):
                if item[i-1] < item[i]:
                    gap = gap + 1
                else:
                    break
            
        gap_change_large_right_continue.append(gap)
        
    table['launch_day_gap_change_large_right_continue'] = gap_change_large_right_continue
       
    return table

In [ ]:
step1 获得聚集的launch_day_list

In [11]:
table123_app_launch_day = agg_list(table123,'app_launch_day')

In [12]:
table123_app_launch_day.head()

,user_id,app_launch_day,launch_gap_list
0,8,"[9.0, 10.0, 11.0, 13.0]","[1.0, 1.0, 2.0]"
1,73,"[2.0, 4.0, 5.0]","[2.0, 1.0]"
2,129,[12.0],[-1]
3,228,[13.0],[-1]
4,351,[3.0],[-1]


In [13]:
table123_app_launch_day.shape,table123.shape

((23923, 3), (79630, 7))

In [ ]:
step2 利用此table提取特征

In [ ]:
特征：
1 最近登录的天数，距离预测间隔的天数是多少？（最近登录的天数）：比如16，转为16-16 = 0
2 登录的天数总计
3 平均登录天数：登录次数 / 从注册日期到最近登录天数（刻画的是充满度）
4 登录间隔：
平均登录间隔（和3作用类似）
最大登录间隔
最小登录间隔
间隔方差（描述波动情况）
间隔的峰度（最大值这个峰的陡峭程度，即是否很尖），间隔的偏度（分布是否对称的偏离程度，0是对称，左右偏分别是正负值）
5 最长连续间隔计算
全区间的最长间隔：记录连续1的个数，方法：用两个标记记录间隔
从尾部开始，往回的最长间隔
两个间隔是否是同一个：如果相等则说明可能是一个（1），不等则很明显（0）
6 最长下降间隔
从尾部开始，往回的间隔是否越来越小的：刻画距离预测时间节点是否越来越稀疏

以上特征：都可以截断时间间隔。
比如：选取注册后一段时间，以及预测前一段时间，分别统计上边这些特征。

In [14]:
time_gap_right = 16 # 区间的最右端
table123_app_launch_day = day_feature_launch(table123_app_launch_day,time_gap_right)

各种特征的shape
23923 23923 23923


In [ ]:
这就获得了这部分的特征了

In [15]:
table123_app_launch_day.head()

,user_id,app_launch_day,launch_gap_list,launch_day_near_pred,launch_day_sum,launch_day_sum_average,launch_day_gap_max,launch_day_gap_ain,launch_day_gap_average,launch_day_gap_std,launch_day_gap_skew,launch_day_gap_kurtosis,launch_day_gap_max_continue,launch_day_gap_max_right_continue,launch_day_gap_max_continue_flag,launch_day_gap_change_large_right_continue
0,8,"[9.0, 10.0, 11.0, 13.0]","[1.0, 1.0, 2.0]",3.0,4,0.80,2.0,1.0,1.333333,0.471405,0.707107,-1.5,2,0,1,1
1,73,"[2.0, 4.0, 5.0]","[2.0, 1.0]",11.0,3,0.75,2.0,1.0,1.500000,0.500000,0.000000,-2.0,1,1,0,0
2,129,[12.0],[-1],4.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,-99.000000,-99.0,-1,-1,-1,-1
3,228,[13.0],[-1],3.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,-99.000000,-99.0,-1,-1,-1,-1
4,351,[3.0],[-1],13.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,-99.000000,-99.0,-1,-1,-1,-1


#### 视频特征

In [ ]:
特征分类：
1 是否发视频
2 day的特征（如上）
3 count的特征：
（参考间隔特征）
最近登录天数的count值
count总值，平均值，最大值，最小值，方差，峰度，偏度,最大值和最小值发生的日子距离预测天的天数
count的最长下降区间：从尾部数，视频量连续下降的区间长度
☆关于最后一个特征：可以增加一个降幅(这段区间的量，占总量的比例)

In [ ]:
用到的函数

In [ ]:
聚集list

In [16]:
def video_pre(table123):
    # 去除无用属性
    del table123['app_launch_day']
    table123 = table123.drop_duplicates()
    table123 = table123.fillna(-1)
    
    # agg聚集list
    table123_video_create_day = table123.groupby('user_id')['video_create_day'].agg(lambda x: list(x)).reset_index()
    table123_video_create_day.columns = ['user_id','video_create_day']
    print(table123_video_create_day.shape)
    
    # 消除多余的-1
    table123_test = table123_video_create_day['video_create_day']
    for i in range(table123_video_create_day.shape[0]):
        temp = table123_video_create_day.iloc[i,1]
        while -1.0 in temp:
            if len(table123_video_create_day.iloc[i,1]) > 1:
                table123_video_create_day.iloc[i,1].remove(-1.0)
            else:
                break
                
    # 增加一列间隔list
    table123_video_create_day['video_day_gap_list'] = -1
    
    list_gap = []
    for item in table123_video_create_day['video_create_day']:
        list_a = []
        if len(item) != 1:
            for i in range(1,len(item)):
                list_a.append(item[i] - item[i-1])
            list_gap.append(list_a)
        else:
            list_gap.append([-1])
    table123_video_create_day['video_day_gap_list'] = list_gap
    

    # 对count特征进行相同的处理
    table123_video_create_day_count = table123.groupby('user_id')['video_create_day_count'].agg(lambda x: list(x)).reset_index()
    table123_video_create_day_count.columns = ['user_id','video_create_day_count']

    table123_test = table123_video_create_day_count['video_create_day_count']
    for i in range(table123_video_create_day_count.shape[0]):
        temp = table123_video_create_day_count.iloc[i,1]
        while -1.0 in temp:
            if len(table123_video_create_day_count.iloc[i,1]) > 1:
                table123_video_create_day_count.iloc[i,1].remove(-1.0)
            else:
                break
    
    # 将这两个merge在一起
    table123_video_create = pd.merge(table123_video_create_day,table123_video_create_day_count,how = 'outer')
    
    return table123_video_create

In [ ]:
step1 获得聚集形态的表格

In [17]:
table123_video_create = video_pre(table123)

(23923, 2)


In [18]:
table123_video_create.head()

,user_id,video_create_day,video_day_gap_list,video_create_day_count
0,8,[-1.0],[-1],[-1.0]
1,73,[-1.0],[-1],[-1.0]
2,129,[-1.0],[-1],[-1.0]
3,228,[-1.0],[-1],[-1.0]
4,351,[-1.0],[-1],[-1.0]


##### 时间特征

In [ ]:
判断是否create video

In [19]:
def add_if_create_video(table123_video_create):
    list_video_create_yes_or_no = [] # 是否发了视频
    for item in table123_video_create['video_create_day']:
        if np.sum(item) == -1.0:
            list_video_create_yes_or_no.append(0) # 表示没发视频
        else:
            list_video_create_yes_or_no.append(1)

    print(len(list_video_create_yes_or_no))

    table123_video_create['if_create_video'] = list_video_create_yes_or_no
    
    return table123_video_create

In [ ]:
提取day的特征

In [20]:
def day_feature_video(table,time_gap_right):
    
    # 天数的特征
    list_max = [] # 最大的天数（最靠近预测日期的天数）
    list_count = [] # 登录天数总计
    list_average = [] # 平均登录天数
    for item in table['video_create_day']:
        if -1.0 in item:
            list_max.append(-1)
            list_count.append(-1)
            list_average.append(-1)
        else:
            list_max.append(time_gap_right - item[-1]) # 取差而不是直接取最近的天数
            list_count.append(len(item))
            list_average.append ( len(item) / (item[-1] - item[0] + 1) )
            # print(item)    
    
    print('各种特征的shape')
    print(len(list_max),len(list_count),len(list_average))
    
    table['video_day_near_pred'] = list_max
    table['video_day_sum'] = list_count
    table['video_day_sum_average'] = list_average
    
    # 间隔的特征
    gap_max = [] # 最大间隔
    gap_min = [] # 最小间隔
    gap_average = [] # 平均间隔
    gap_std = [] # 间隔方差
    gap_skew = [] # 间隔的偏度：衡量分布是左偏还是右偏，小于0左偏，说明过了最高值，下降的比上升快（更有可能弃坑）
    gap_kurtosis = [] # 间隔的峰度：大于0说明尖峰是陡升的，小于0则说明爬升较缓和
    for item in table['video_day_gap_list']:
        if -1 not in item:            
            gap_max.append(np.max(item))
            gap_min.append(np.min(item))
            gap_average.append(np.mean(item))
            gap_std.append(np.std(item))
            gap_skew.append(stats.skew(item))
            gap_kurtosis.append(stats.kurtosis(item))
        else:
            gap_max.append(-1)
            gap_min.append(-1)
            gap_average.append(-1)
            gap_std.append(-1)
            # 考虑到取值范围，调整
            gap_skew.append(-99)
            gap_kurtosis.append(-99)
    
    table['video_day_gap_max'] = gap_max
    table['video_day_gap_ain'] = gap_min
    table['video_day_gap_average'] = gap_average
    table['video_day_gap_std'] = gap_std
    table['video_day_gap_skew'] = gap_skew
    table['video_day_gap_kurtosis'] = gap_kurtosis
    
    # 连续间隔特征
    gap_max_continue = [] # 全区间的最大连续间隔：注意1代表连续2天都有记录（注意1和2是差1的！），0代表没有连续的
    for item in table['video_day_gap_list']:
        gap = -1 # =0代表没有1出现，即没有连续x天的记录。-1代表没有间隔
        if -1 not in item:
            gap = 0
            # 只有有1才可能gap不等于0
            if 1 in item:
                gap = 1 # 1的个数，至少1个
                left = 0
                right = 0
                switch = 0 # 用于控制间隔是否开始的开关
                # [2 3 1 1 2 3 1 1 1]
                for i in range(len(item)):           
                    # 开始计算间隔长度
                    if switch == 1 and item[i] == 1:
                        right = i
                    # 到1的新出发点
                    elif switch == 0 and item[i] == 1:
                        switch = 1
                        left = i
                        right = i
                    # 进入新的间隔停止期，遭遇非1，计算间隔长度
                    elif switch == 1 and item[i] != 1:
                        switch = 0
                        temp = right - left + 1
                        # 只记录最大的gap
                        if temp > gap:
                            gap = temp
                    # 进入非1的区间，不需要做任何事
                    # switch == 0 and item[i] != 1
                    else:
                        continue
                    
                     # 必须考虑是不是最后一个值了，此时对于是否是1，有不同的处理
                    if i == len(item) - 1:
                        # 此时需要计算一遍gap
                        if item[i] == 1:
                            temp = right - left + 1
                            # 只记录最大的gap
                            if temp > gap:
                                gap = temp
                                
        gap_max_continue.append(gap)       
        
    # 从后往前数的最长连续记录
    gap_max_right_continue = []
    for item in table['video_day_gap_list']:
        gap = -1 # 没有间隔只能= -1
        if -1 not in item:
            gap = 0
            # 最后一个值是1才往前走
            if item[-1] == 1:
                gap = 1 # 1的个数
                # 从后往前的循环，从倒数第2个开始
                for i in range(len(item)-2,-1,-1):
                    if item[i] == 1:
                        gap = gap + 1
                    else:
                        break
        gap_max_right_continue.append(gap)
    
    # 比较两个gap是否数量相同，作为第3个特征，相同 = 0，不同 = 1（不区分左右，没有明显的意义）
    gap_continue_flag = []
    for i in range(len(gap_max_continue)):
        if gap_max_continue[i] == -1 or gap_max_right_continue[i] == -1:
            gap_continue_flag.append(-1) # 没有间隔
        elif gap_max_continue[i] == gap_max_right_continue[i]:
            gap_continue_flag.append(0)
        else:
            gap_continue_flag.append(1)
    
    table['video_day_gap_max_continue'] = gap_max_continue
    table['video_day_gap_max_right_continue'] = gap_max_right_continue
    table['video_day_gap_max_continue_flag'] = gap_continue_flag
    
    
    # 从后往前数的，序列变得越来越稀疏的长度：即正向看的话，间隔值是否越来越大
    gap_change_large_right_continue = []
    for item in table['video_day_gap_list']:
        gap = -1
        if -1 not in item:
            gap = 0
            for i in range(len(item)-1,0,-1):
                if item[i-1] < item[i]:
                    gap = gap + 1
                else:
                    break
            
        gap_change_large_right_continue.append(gap)
        
    table['video_day_gap_change_large_right_continue'] = gap_change_large_right_continue
       
    return table

In [ ]:
step1 是否create video的特征

In [21]:
table123_video_create = add_if_create_video(table123_video_create)

23923


In [22]:
table123_video_create.head()

,user_id,video_create_day,video_day_gap_list,video_create_day_count,if_create_video
0,8,[-1.0],[-1],[-1.0],0
1,73,[-1.0],[-1],[-1.0],0
2,129,[-1.0],[-1],[-1.0],0
3,228,[-1.0],[-1],[-1.0],0
4,351,[-1.0],[-1],[-1.0],0


In [ ]:
step2 提取day的特征

In [ ]:
video_create_day的单独特征：
参考launch_day

In [23]:
time_gap_right = 16 # 区间的最右端
table123_video_create = day_feature_video(table123_video_create,time_gap_right)

各种特征的shape
23923 23923 23923


In [26]:
table123_video_create.iloc[310:316,:]

,user_id,video_create_day,video_day_gap_list,video_create_day_count,if_create_video,video_day_near_pred,video_day_sum,video_day_sum_average,video_day_gap_max,video_day_gap_ain,video_day_gap_average,video_day_gap_std,video_day_gap_skew,video_day_gap_kurtosis,video_day_gap_max_continue,video_day_gap_max_right_continue,video_day_gap_max_continue_flag,video_day_gap_change_large_right_continue
310,19462,[-1.0],[-1],[-1.0],0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1,-1,-1,-1
311,19526,[-1.0],[-1],[-1.0],0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1,-1,-1,-1
312,19629,[-1.0],[-1],[-1.0],0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1,-1,-1,-1
313,19682,[-1.0],[-1],[-1.0],0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1,-1,-1,-1
314,19727,[-1.0],[-1],[-1.0],0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1,-1,-1,-1
315,19805,"[8.0, 9.0]",[1.0],"[2.0, 1.0]",1,7.0,2,1.0,1.0,1.0,1.0,0.0,0.0,-3.0,1,1,0,0


##### 数量特征

In [27]:
def count_feature_video(table123_video_create,time_gap_right):
    
    count_pred = [] # 最近天数的count数量
    count_sum = [] # 总量
    count_average = [] # 均值（和创建视频天数挂钩，不是单纯的平均值）
    count_max = [] # 最大值
    count_day_max = [] # 最大值距离预测day的天数
    count_min = [] # 最小值
    count_day_min = [] # 最小值距离预测day的天数
    count_std = [] # 方差
    count_skew = [] # 偏度
    count_kurtosis = [] # 峰度    

    for i in range(table123_video_create.shape[0]):
    # for i in range(10):
        if_create = table123_video_create.iloc[i,4]
        item = table123_video_create.iloc[i,3] 
        item_day = table123_video_create.iloc[i,1] 
        if if_create == 0: # 没有发视频
            count_pred.append(-1)
            count_sum.append(-1) 
            count_average.append(-1) 
            count_max.append(-1) 
            count_day_max.append(-1) 
            count_min.append(-1) 
            count_day_min.append(-1) 
            count_std.append(-1) 
            count_skew.append(-99) 
            count_kurtosis.append(-99) 
        else:
            count_pred.append(item[-1])
            count_sum.append(np.sum(item)) 
            count_average.append(np.sum(item) / (item_day[-1] - item_day[0] + 1) )
            count_max.append(np.max(item)) 
            max_index = item.index(np.max(item))
            count_day_max.append(time_gap_right - item_day[max_index]) 
            count_min.append(np.min(item)) 
            min_index = item.index(np.min(item))
            count_day_min.append(time_gap_right - item_day[min_index]) 
            count_std.append(np.std(item)) 
            count_skew.append(stats.skew(item)) 
            count_kurtosis.append(stats.kurtosis(item))   
            
    table123_video_create['video_count_pred'] = count_pred
    table123_video_create['video_count_sum'] = count_sum
    table123_video_create['video_count_average'] = count_average 
    table123_video_create['video_count_max'] = count_max 
    table123_video_create['video_count_day_max'] = count_day_max
    table123_video_create['video_count_min'] = count_min
    table123_video_create['video_count_day_min'] = count_day_min 
    table123_video_create['video_count_std'] = count_std
    table123_video_create['video_count_skew'] = count_skew 
    table123_video_create['video_count_kurtosis'] = count_kurtosis
    
    
    # 从后往前数的，序列值越来越小的长度。衡量稀疏度
    gap_change_small_right_continue = []
    for item in table123_video_create['video_create_day_count']:
        gap = -1
        if -1 not in item:
            gap = 0
            for i in range(len(item)-1,0,-1):
                if item[i-1] > item[i]:
                    gap = gap + 1
                else:
                    break
            
        gap_change_small_right_continue.append(gap)
        
    table123_video_create['video_count_gap_change_small_right_continue'] = gap_change_small_right_continue 
    
    return table123_video_create

In [ ]:
step3 提取有关数量的特征

In [28]:
time_gap_right = 16 # 区间的最右端
table123_video_create = count_feature_video(table123_video_create,time_gap_right)

In [29]:
table123_video_create.iloc[11:16,:]

,user_id,video_create_day,video_day_gap_list,video_create_day_count,if_create_video,video_day_near_pred,video_day_sum,video_day_sum_average,video_day_gap_max,video_day_gap_ain,...,video_count_sum,video_count_average,video_count_max,video_count_day_max,video_count_min,video_count_day_min,video_count_std,video_count_skew,video_count_kurtosis,video_count_gap_change_small_right_continue
11,948,[-1.0],[-1],[-1.0],0,-1.0,-1,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1
12,1108,[-1.0],[-1],[-1.0],0,-1.0,-1,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1
13,1115,[-1.0],[-1],[-1.0],0,-1.0,-1,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1
14,1197,[-1.0],[-1],[-1.0],0,-1.0,-1,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1
15,1341,[-1.0],[-1],[-1.0],0,-1.0,-1,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1


#### 合并登录特征和视频特征

In [30]:
table123_feature = pd.merge(
    table123_app_launch_day,table123_video_create,how = 'outer')

In [31]:
table123_app_launch_day.shape,table123_video_create.shape,table123_feature.shape

((23923, 16), (23923, 29), (23923, 44))

In [32]:
table123_feature.head()

,user_id,app_launch_day,launch_gap_list,launch_day_near_pred,launch_day_sum,launch_day_sum_average,launch_day_gap_max,launch_day_gap_ain,launch_day_gap_average,launch_day_gap_std,...,video_count_sum,video_count_average,video_count_max,video_count_day_max,video_count_min,video_count_day_min,video_count_std,video_count_skew,video_count_kurtosis,video_count_gap_change_small_right_continue
0,8,"[9.0, 10.0, 11.0, 13.0]","[1.0, 1.0, 2.0]",3.0,4,0.80,2.0,1.0,1.333333,0.471405,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1
1,73,"[2.0, 4.0, 5.0]","[2.0, 1.0]",11.0,3,0.75,2.0,1.0,1.500000,0.500000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1
2,129,[12.0],[-1],4.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1
3,228,[13.0],[-1],3.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1
4,351,[3.0],[-1],13.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1


In [34]:
table123_feature.to_csv(
    'dataset_B_for_model_method/temp_dataset/table123_1_16_first_3_feature.csv',index = False)

#### 行为特征

In [ ]:
读取table123_1_16_feature

In [38]:
table123_feature = pd.read_csv(
    'dataset_B_for_model_method/temp_dataset/table123_1_16_first_3_feature.csv')

In [39]:
table123_feature.head()

,user_id,app_launch_day,launch_gap_list,launch_day_near_pred,launch_day_sum,launch_day_sum_average,launch_day_gap_max,launch_day_gap_ain,launch_day_gap_average,launch_day_gap_std,...,video_count_sum,video_count_average,video_count_max,video_count_day_max,video_count_min,video_count_day_min,video_count_std,video_count_skew,video_count_kurtosis,video_count_gap_change_small_right_continue
0,8,"[9.0, 10.0, 11.0, 13.0]","[1.0, 1.0, 2.0]",3.0,4,0.80,2.0,1.0,1.333333,0.471405,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1
1,73,"[2.0, 4.0, 5.0]","[2.0, 1.0]",11.0,3,0.75,2.0,1.0,1.500000,0.500000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1
2,129,[12.0],[-1],4.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1
3,228,[13.0],[-1],3.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1
4,351,[3.0],[-1],13.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-99.0,-99.0,-1


In [40]:
act.head()

,user_id,activity_day,activity_page,video_id,author_id,action_type,activity_count
0,8,9,3,1955966,309219,0,2
1,8,9,0,1340816,12378,0,1
2,8,9,0,3488372,107008,0,1
3,8,9,0,1705402,37639,0,2
4,8,9,0,3710937,1312718,0,1


In [ ]:
特征总结：
1 day单独特征（如上）
2 day和count的特征（如上）
☆对count还可以进行更精细的挖掘：比如count的diff（变化量）的list
——决定加入：
统计：总量，均值，正负值的max和day（day往后一天算），std，偏度，峰度
是否是连减，是否降低量增加

3 关于page和type
原有的：每种page占活跃总量的比例
☆可以参考的特征：每种page和type随day的变化list的上述统计值
4 video和author
list：将每个author的对应的video观看量作为list，
统计：
author的数量，video的总量 / author的数量：
这两个实际可以看做author_video_count_list的len，average，sum（前边count统计过了）
average的意义——用户偏好：author数量少，video数量又大的。video / count的值
其余的统计量：
max，max占sum的比例值
std：衡量波动情况，看的作者是否杂，杂的user没有关注的用户，容易弃坑
峰度和偏度：刻画list的变化趋势（需要先给list排序）
用户偏好：author数量少，video数量又大的。video / count的值
☆可以继续挖掘的特征：重要author的count，随着day的变化list（上边统计的是所有author的总值）
——上边【所有的】，都是统计的单一特征，而这个是多元的：author（count） + day
☆是否是heat_author：创建过视频的user_id，创建视频的互动量(是否是top之内的)

In [ ]:
activity_day 时间特征同上
activity_count 用户行为的计数/天 
activity_page：“关注页”、”个人主页“、”发现页“、”同城页“或”其他页“中的一个，编号0-4
activity_type：“播放“、”关注“、”点赞“、”转发“、”举报“和”减少此类作品“中的一个，编号0-5
video_id和author_id：可以统计id的信息，比如关注video_id中的高量播放视频，关注author_id是不是这批新注册的id

##### 时间特征

In [ ]:
使用到的函数

In [ ]:
预处理函数

In [ ]:
注意：count的gap有负值，因此null不能用-1代替！只能用np.nan或者非数值。

In [41]:
def activity_day_count_pre(act):
    # 在activity_day和activity_count的特征统计时，不需要中间这几个特征，因此可以先去掉
    drop_columns = ['activity_page', 'video_id', 'author_id','action_type']
    act.drop(drop_columns,axis = 1 ,inplace = True)
    
    # 获得count指标（按照day计算sum）
    activity_count = act.groupby(['user_id','activity_day']).sum()
    activity_count.reset_index(inplace = True)
    
    print(activity_count.shape)

    # 用activity_count作为提取day和count的特征
    activity_day_temp = activity_count.groupby('user_id')['activity_day'].agg(lambda x: list(x)).reset_index()
    activity_day_temp.columns = ['user_id','activity_day']
    
    # day和count都增加了间隔list
    # 增加一列间隔list
    activity_day_temp['act_day_gap_list'] = -1
    
    list_gap = []
    for item in activity_day_temp['activity_day']:
        list_a = []
        if len(item) != 1:
            for i in range(1,len(item)):
                list_a.append(item[i] - item[i-1])
            list_gap.append(list_a)
        else:
            list_gap.append([-1])
    activity_day_temp['act_day_gap_list'] = list_gap

    activity_count_temp = activity_count.groupby('user_id')['activity_count'].agg(lambda x: list(x)).reset_index()
    activity_count_temp.columns = ['user_id','activity_count']
    
    
    # 增加一列间隔list
    activity_count_temp['act_count_gap_list'] = -1
    
    list_gap = []
    for item in activity_count_temp['activity_count']:
        list_a = []
        if len(item) != 1:
            for i in range(1,len(item)):
                list_a.append(item[i] - item[i-1])
            list_gap.append(list_a)
        else:
            list_gap.append(np.nan) # 由于count的差值可以是负的，因此-1不能用了！
    activity_count_temp['act_count_gap_list'] = list_gap
    
    print(activity_day_temp.shape,activity_count_temp.shape)
    
    # 合并一下两者
    activity_day_and_count = pd.merge(activity_day_temp,activity_count_temp,how = 'outer')
    
    return activity_day_and_count

In [ ]:
提取day特征

In [42]:
def day_feature_act(table,time_gap_right):
    
    # 天数的特征
    list_max = [] # 最大的天数（最靠近预测日期的天数）
    list_count = [] # 登录天数总计
    list_average = [] # 平均登录天数
    for item in table['activity_day']:
        if -1.0 in item:
            list_max.append(-1)
            list_count.append(-1)
            list_average.append(-1)
        else:
            list_max.append(time_gap_right - item[-1]) # 取差而不是直接取最近的天数
            list_count.append(len(item))
            list_average.append ( len(item) / (item[-1] - item[0] + 1) )
            # print(item)    
    
    print('各种特征的shape')
    print(len(list_max),len(list_count),len(list_average))
    
    table['act_day_near_pred'] = list_max
    table['act_day_sum'] = list_count
    table['act_day_sum_average'] = list_average
    
    # 间隔的特征
    gap_max = [] # 最大间隔
    gap_min = [] # 最小间隔
    gap_average = [] # 平均间隔
    gap_std = [] # 间隔方差
    gap_skew = [] # 间隔的偏度：衡量分布是左偏还是右偏，小于0左偏，说明过了最高值，下降的比上升快（更有可能弃坑）
    gap_kurtosis = [] # 间隔的峰度：大于0说明尖峰是陡升的，小于0则说明爬升较缓和
    for item in table['act_day_gap_list']:
        if -1 not in item:            
            gap_max.append(np.max(item))
            gap_min.append(np.min(item))
            gap_average.append(np.mean(item))
            gap_std.append(np.std(item))
            gap_skew.append(stats.skew(item))
            gap_kurtosis.append(stats.kurtosis(item))
        else:
            gap_max.append(-1)
            gap_min.append(-1)
            gap_average.append(-1)
            gap_std.append(-1)
            # 考虑到取值范围，调整
            gap_skew.append(-99)
            gap_kurtosis.append(-99)
    
    table['act_day_gap_max'] = gap_max
    table['act_day_gap_ain'] = gap_min
    table['act_day_gap_average'] = gap_average
    table['act_day_gap_std'] = gap_std
    table['act_day_gap_skew'] = gap_skew
    table['act_day_gap_kurtosis'] = gap_kurtosis
    
    # 连续间隔特征
    gap_max_continue = [] # 全区间的最大连续间隔：注意1代表连续2天都有记录（注意1和2是差1的！），0代表没有连续的
    for item in table['act_day_gap_list']:
        gap = -1 # =0代表没有1出现，即没有连续x天的记录。-1代表没有间隔
        if -1 not in item:
            gap = 0
            # 只有有1才可能gap不等于0
            if 1 in item:
                gap = 1 # 1的个数，至少1个
                left = 0
                right = 0
                switch = 0 # 用于控制间隔是否开始的开关
                # [2 3 1 1 2 3 1 1 1]
                for i in range(len(item)):           
                    # 开始计算间隔长度
                    if switch == 1 and item[i] == 1:
                        right = i
                    # 到1的新出发点
                    elif switch == 0 and item[i] == 1:
                        switch = 1
                        left = i
                        right = i
                    # 进入新的间隔停止期，遭遇非1，计算间隔长度
                    elif switch == 1 and item[i] != 1:
                        switch = 0
                        temp = right - left + 1
                        # 只记录最大的gap
                        if temp > gap:
                            gap = temp
                    # 进入非1的区间，不需要做任何事
                    # switch == 0 and item[i] != 1
                    else:
                        continue
                    
                     # 必须考虑是不是最后一个值了，此时对于是否是1，有不同的处理
                    if i == len(item) - 1:
                        # 此时需要计算一遍gap
                        if item[i] == 1:
                            temp = right - left + 1
                            # 只记录最大的gap
                            if temp > gap:
                                gap = temp
                                
        gap_max_continue.append(gap)       
        
    # 从后往前数的最长连续记录
    gap_max_right_continue = []
    for item in table['act_day_gap_list']:
        gap = -1 # 没有间隔只能= -1
        if -1 not in item:
            gap = 0
            # 最后一个值是1才往前走
            if item[-1] == 1:
                gap = 1 # 1的个数
                # 从后往前的循环，从倒数第2个开始
                for i in range(len(item)-2,-1,-1):
                    if item[i] == 1:
                        gap = gap + 1
                    else:
                        break
        gap_max_right_continue.append(gap)
    
    # 比较两个gap是否数量相同，作为第3个特征，相同 = 0，不同 = 1（不区分左右，没有明显的意义）
    gap_continue_flag = []
    for i in range(len(gap_max_continue)):
        if gap_max_continue[i] == -1 or gap_max_right_continue[i] == -1:
            gap_continue_flag.append(-1) # 没有间隔
        elif gap_max_continue[i] == gap_max_right_continue[i]:
            gap_continue_flag.append(0)
        else:
            gap_continue_flag.append(1)
    
    table['act_day_gap_max_continue'] = gap_max_continue
    table['act_day_gap_max_right_continue'] = gap_max_right_continue
    table['act_day_gap_max_continue_flag'] = gap_continue_flag
    
    
    # 从后往前数的，序列变得越来越稀疏的长度：即正向看的话，间隔值是否越来越大
    gap_change_large_right_continue = []
    for item in table['act_day_gap_list']:
        gap = -1
        if -1 not in item:
            gap = 0
            for i in range(len(item)-1,0,-1):
                if item[i-1] < item[i]:
                    gap = gap + 1
                else:
                    break
            
        gap_change_large_right_continue.append(gap)
        
    table['act_day_gap_change_large_right_continue'] = gap_change_large_right_continue
       
    return table


In [ ]:
step1 仍然是先转换基本数据集（在时间特征和数量特征部分使用）

In [43]:
act_day_and_count = activity_day_count_pre(act)

(68554, 3)
(20535, 3) (20535, 3)


In [44]:
act_day_and_count.head()

,user_id,activity_day,act_day_gap_list,activity_count,act_count_gap_list
0,8,"[9, 10, 11, 13]","[1, 1, 2]","[21, 39, 5, 31]","[18, -34, 26]"
1,73,"[4, 5]",[1],"[5, 29]",[24]
2,129,[12],[-1],[1],NaN
3,228,[13],[-1],[22],NaN
4,351,[3],[-1],[176],NaN


In [ ]:
step2 获得day特征

In [45]:
time_gap_right = 16 # 区间的最右端
act_day_and_count = day_feature_act(act_day_and_count,time_gap_right)

各种特征的shape
20535 20535 20535


In [46]:
act_day_and_count.head()

,user_id,activity_day,act_day_gap_list,activity_count,act_count_gap_list,act_day_near_pred,act_day_sum,act_day_sum_average,act_day_gap_max,act_day_gap_ain,act_day_gap_average,act_day_gap_std,act_day_gap_skew,act_day_gap_kurtosis,act_day_gap_max_continue,act_day_gap_max_right_continue,act_day_gap_max_continue_flag,act_day_gap_change_large_right_continue
0,8,"[9, 10, 11, 13]","[1, 1, 2]","[21, 39, 5, 31]","[18, -34, 26]",3,4,0.8,2,1,1.333333,0.471405,0.707107,-1.5,2,0,1,1
1,73,"[4, 5]",[1],"[5, 29]",[24],11,2,1.0,1,1,1.000000,0.000000,0.000000,-3.0,1,1,0,0
2,129,[12],[-1],[1],NaN,4,1,1.0,-1,-1,-1.000000,-1.000000,-99.000000,-99.0,-1,-1,-1,-1
3,228,[13],[-1],[22],NaN,3,1,1.0,-1,-1,-1.000000,-1.000000,-99.000000,-99.0,-1,-1,-1,-1
4,351,[3],[-1],[176],NaN,13,1,1.0,-1,-1,-1.000000,-1.000000,-99.000000,-99.0,-1,-1,-1,-1


##### 数量特征 

In [ ]:
提取count的函数
多了对count的diff后的特征统计

In [47]:
def count_feature_act(act_day_and_count,time_gap_right):
    
    # 先是count list
    count_pred = [] # 最近天数的count数量
    count_sum = [] # 总量
    count_average = [] # 均值（和创建视频天数挂钩，不是单纯的平均值）
    count_max = [] # 最大值
    count_day_max = [] # 最大值距离预测day的天数
    count_min = [] # 最小值
    count_day_min = [] # 最小值距离预测day的天数
    count_std = [] # 方差
    count_skew = [] # 偏度
    count_kurtosis = [] # 峰度    

    for i in range(act_day_and_count.shape[0]):
    # for i in range(10):
        item = act_day_and_count.iloc[i,3] 
        item_day = act_day_and_count.iloc[i,1] 
        
        count_pred.append(item[-1])
        count_sum.append(np.sum(item)) 
        count_average.append(np.sum(item) / (item_day[-1] - item_day[0] + 1) )
        count_max.append(np.max(item)) 
        max_index = item.index(np.max(item))
        count_day_max.append(time_gap_right - item_day[max_index]) 
        count_min.append(np.min(item)) 
        min_index = item.index(np.min(item))
        count_day_min.append(time_gap_right - item_day[min_index])         
        count_std.append(np.std(item)) 
        count_skew.append(stats.skew(item)) 
        count_kurtosis.append(stats.kurtosis(item))   
            
    act_day_and_count['act_count_pred'] = count_pred
    act_day_and_count['act_count_sum'] = count_sum
    act_day_and_count['act_count_average'] = count_average 
    act_day_and_count['act_count_max'] = count_max 
    act_day_and_count['act_count_day_max'] = count_day_max
    act_day_and_count['act_count_min'] = count_min
    act_day_and_count['act_count_day_min'] = count_day_min 
    act_day_and_count['act_count_std'] = count_std
    act_day_and_count['act_count_skew'] = count_skew 
    act_day_and_count['act_count_kurtosis'] = count_kurtosis
    
    
    # 从后往前数的，序列值越来越小的长度。衡量稀疏度
    gap_change_small_right_continue = []
    for item in act_day_and_count['activity_count']:
        gap = -1
        if -1 not in item:
            gap = 0
            for i in range(len(item)-1,0,-1):
                if item[i-1] > item[i]:
                    gap = gap + 1
                else:
                    break
            
        gap_change_small_right_continue.append(gap)
        
    act_day_and_count['act_count_gap_change_small_right_continue'] = gap_change_small_right_continue 
    
    # 然后是count diff序列的统计特征
    
    count_diff_sum = [] # 总量
    count_diff_average = [] # 均值（和创建视频天数挂钩，不是单纯的平均值）
    count_diff_minus_max = [] # 负值中的最大值（衡量的是降低的程度）
    count_diff_day_minus_max = [] # 最大值距离预测day的天数
    count_diff_plus_max = [] # 正值中最大值（衡量增加程度）
    count_diff_day_plus_max = []
    count_diff_std = [] # 方差
    count_diff_skew = [] # 偏度
    count_diff_kurtosis = [] # 峰度    

    for i in range(act_day_and_count.shape[0]):
    # for i in range(10):
        item = act_day_and_count.iloc[i,4] # 这次取4不是3 
        item_day = act_day_and_count.iloc[i,1] 
        item_count = act_day_and_count.iloc[i,3] # 用于判断是否没有间隔
        
        # 没有间隔的情况
        if len(item_count) == 1:
            
            count_diff_sum.append(np.nan) 
            count_diff_average.append(np.nan)
            count_diff_minus_max.append(np.nan) 
            count_diff_day_minus_max.append(np.nan)

            count_diff_plus_max.append(np.nan) 
            count_diff_day_plus_max.append(np.nan)

            count_diff_std.append(np.nan)
            count_diff_skew.append(np.nan) 
            count_diff_kurtosis.append(np.nan)   
            
        else:
            
            count_diff_sum.append(np.sum(item)) 
            count_diff_average.append(np.sum(item) /  len(item) ) # 对于gap用len（item）本身长度更合适  

            temp = []
            for i in range(len(item)):
                if item[i] > 0:
                    temp.append(0)
                else:
                    temp.append(abs(item[i]))

            count_diff_minus_max.append(np.max(temp)) 
            max_index = temp.index(np.max(temp)) + 1 # 取间隔的后者
            count_diff_day_minus_max.append(time_gap_right - item_day[max_index]) 

            count_diff_plus_max.append(np.max(item)) 
            max_index = item.index(np.max(item)) + 1
            count_diff_day_plus_max.append(time_gap_right - item_day[max_index]) 

            count_diff_std.append(np.std(item)) 
            count_diff_skew.append(stats.skew(item)) 
            count_diff_kurtosis.append(stats.kurtosis(item))   
            
    act_day_and_count['act_count_diff_sum'] = count_diff_sum
    act_day_and_count['act_count_diff_average'] = count_diff_average 
    act_day_and_count['act_count_diff_minus_max'] = count_diff_minus_max 
    act_day_and_count['act_count_diff_day_minus_max'] = count_diff_day_minus_max
    act_day_and_count['act_count_diff_plus_max'] = count_diff_plus_max
    act_day_and_count['act_count_diff_day_plus_max'] = count_diff_day_plus_max
    act_day_and_count['act_count_diff_std'] = count_diff_std
    act_day_and_count['act_count_diff_skew'] = count_diff_skew 
    act_day_and_count['act_count_diff_kurtosis'] = count_diff_kurtosis    
    
    # diff的连续特征：连减持续时间长度，降低量连续增加持续长度
    diff_minus_right_continue = []
    diff_change_small_right_continue = []
    for i in range(act_day_and_count.shape[0]):
    # for i in range(10):
        item = act_day_and_count.iloc[i,4] # 这次取4不是3 
        item_count = act_day_and_count.iloc[i,3] # 用于判断是否没有间隔

        gap_minus = -1
        gap_small = -1
        # 有间隔才可以
        if len(item_count) > 1:
            gap_minus = 0
            gap_small = 0
            
            # minus
            for j in range(len(item)-1,-1,-1):
                if item[j] < 0:
                    gap_minus = gap_minus + 1
                else:
                    break
            
            for j in range(len(item)-1,0,-1):
                if item[j-1] > item[j]:
                    gap_small = gap_small + 1
                else:
                    break
            
        diff_minus_right_continue.append(gap_minus)
        diff_change_small_right_continue.append(gap_small)
        
    act_day_and_count['act_count_diff_minus_right_continue'] = diff_minus_right_continue
    act_day_and_count['act_count_gap_diff_change_small_right_continue'] = diff_change_small_right_continue
    
    return act_day_and_count

In [ ]:
step1 获取count特征

In [48]:
time_gap_right = 16 # 区间的最右端
act_day_and_count = count_feature_act(act_day_and_count,time_gap_right)

In [49]:
act_day_and_count.head()

,user_id,activity_day,act_day_gap_list,activity_count,act_count_gap_list,act_day_near_pred,act_day_sum,act_day_sum_average,act_day_gap_max,act_day_gap_ain,...,act_count_diff_average,act_count_diff_minus_max,act_count_diff_day_minus_max,act_count_diff_plus_max,act_count_diff_day_plus_max,act_count_diff_std,act_count_diff_skew,act_count_diff_kurtosis,act_count_diff_minus_right_continue,act_count_gap_diff_change_small_right_continue
0,8,"[9, 10, 11, 13]","[1, 1, 2]","[21, 39, 5, 31]","[18, -34, 26]",3,4,0.8,2,1,...,3.333333,34.0,5.0,26.0,3.0,26.599916,-0.65944,-1.5,0,0
1,73,"[4, 5]",[1],"[5, 29]",[24],11,2,1.0,1,1,...,24.000000,0.0,11.0,24.0,11.0,0.000000,0.00000,-3.0,0,0
2,129,[12],[-1],[1],NaN,4,1,1.0,-1,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
3,228,[13],[-1],[22],NaN,3,1,1.0,-1,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
4,351,[3],[-1],[176],NaN,13,1,1.0,-1,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1


##### page和type

In [ ]:
上边动了原始的table，因此要重读

In [50]:
act = pd.read_csv('dataset_B_for_model_method/split_origin_dataset/act_1_16.csv')

In [51]:
act.head()

,user_id,activity_day,activity_page,video_id,author_id,action_type,activity_count
0,8,9,3,1955966,309219,0,2
1,8,9,0,1340816,12378,0,1
2,8,9,0,3488372,107008,0,1
3,8,9,0,1705402,37639,0,2
4,8,9,0,3710937,1312718,0,1


In [ ]:
activity_page和activity_type
由于每种行为和页面也有区分度，那么将这些行为分成各个独立的特征比较好。
page共5种，type共6种，即11种基础特征。
每种基础特征下的特征有：
活跃总量的占比（关注页占所有活跃总量的比例）：——计算的时候必须联合activity_count，因为如果=2的话，这个页面要算2次的。

In [52]:
def page_feature(act):
    # 获得每种page的数量（count）
    act_page_sum = act.groupby(['user_id','activity_page'])['activity_count'].sum()
    act_page_sum = act_page_sum.reset_index()
    
    # 获得每个user的总体page数量
    act_all_page_sum = act_page_sum.groupby('user_id')['activity_count'].sum()
    act_all_page_sum = act_all_page_sum.reset_index()
    act_all_page_sum.columns = ['user_id','activity_count_sum']
    
    # 链接这两个参数，并计算比例
    act_page_sum_new = pd.merge(act_page_sum,act_all_page_sum,how ='outer')
    act_page_sum_new['page_ratio'] = act_page_sum_new['activity_count'] / act_page_sum_new['activity_count_sum']
    
    # 用完可以扔了
    del act_page_sum_new['activity_count']
    del act_page_sum_new['activity_count_sum']

    # 通过pivot_table将5种数值散开
    pivot_page = pd.pivot_table(act_page_sum_new,index = 'user_id',columns = 'activity_page',
                                values = 'page_ratio')
    pivot_page.columns = ['activity_page_0','activity_page_1','activity_page_2',
                          'activity_page_3','activity_page_4']
    pivot_page = pivot_page.reset_index()
    # 没有的项用0填充
    pivot_page = pivot_page.fillna(0)
    
    return pivot_page

In [53]:
def type_feature(act):
    
    act_type_sum = act.groupby(['user_id','action_type'])['activity_count'].sum()
    act_type_sum = act_type_sum.reset_index()

    act_all_type_sum = act_type_sum.groupby('user_id')['activity_count'].sum()
    act_all_type_sum = act_all_type_sum.reset_index()
    act_all_type_sum.columns = ['user_id','activity_count_sum']

    act_type_sum_new = pd.merge(act_type_sum,act_all_type_sum,how ='outer')
    act_type_sum_new['type_ratio'] = act_type_sum_new['activity_count'] / act_type_sum_new['activity_count_sum']

    del act_type_sum_new['activity_count']
    del act_type_sum_new['activity_count_sum']

    pivot_type = pd.pivot_table(act_type_sum_new,index = 'user_id',columns = 'action_type',values = 'type_ratio')
    pivot_type.columns = ['action_type_0','action_type_1','action_type_2','action_type_3','action_type_4','action_type_5']
    pivot_type = pivot_type.reset_index()
    pivot_type = pivot_type.fillna(0)
    
    return pivot_type

In [ ]:
先是page

In [54]:
act_pivot_page = page_feature(act)

In [55]:
act_pivot_page.head()

,user_id,activity_page_0,activity_page_1,activity_page_2,activity_page_3,activity_page_4
0,8,0.770833,0.041667,0.000000,0.187500,0.000000
1,73,0.617647,0.000000,0.000000,0.382353,0.000000
2,129,1.000000,0.000000,0.000000,0.000000,0.000000
3,228,0.000000,0.000000,0.000000,1.000000,0.000000
4,351,0.431818,0.011364,0.005682,0.528409,0.022727


In [ ]:
然后是type

In [56]:
act_pivot_type = type_feature(act)

In [57]:
act_pivot_type.head()

,user_id,action_type_0,action_type_1,action_type_2,action_type_3,action_type_4,action_type_5
0,8,0.979167,0.020833,0.000000,0.0,0.0,0.0
1,73,0.941176,0.058824,0.000000,0.0,0.0,0.0
2,129,1.000000,0.000000,0.000000,0.0,0.0,0.0
3,228,1.000000,0.000000,0.000000,0.0,0.0,0.0
4,351,0.988636,0.005682,0.005682,0.0,0.0,0.0


In [ ]:
最后merge到一起即可

In [58]:
act_page_and_type = pd.merge(act_pivot_page,act_pivot_type,how = 'outer')

In [59]:
act_page_and_type.head()

,user_id,activity_page_0,activity_page_1,activity_page_2,activity_page_3,activity_page_4,action_type_0,action_type_1,action_type_2,action_type_3,action_type_4,action_type_5
0,8,0.770833,0.041667,0.000000,0.187500,0.000000,0.979167,0.020833,0.000000,0.0,0.0,0.0
1,73,0.617647,0.000000,0.000000,0.382353,0.000000,0.941176,0.058824,0.000000,0.0,0.0,0.0
2,129,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.0
3,228,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.0
4,351,0.431818,0.011364,0.005682,0.528409,0.022727,0.988636,0.005682,0.005682,0.0,0.0,0.0


In [60]:
act_page_and_type.shape

(20535, 12)

##### video和author

In [ ]:
4 video和author
用户偏好：author数量少，video数量又大的。video / count的值
list：将每个author的对应的video观看量作为list，
统计：mean，std，max，min，最大值max对应的author是否是热度author（video创建量极高的，比如top1000）
对于这个list，先排序，排完序后统计峰度（观察峰值变化）

关于原有的热度特征的处理：
count值不要了。
是否是heat_author：变成自己create的video，被互动量的占比（占全部的，可以是author的名次，也可以是视频量的占比）

In [61]:
def author_id_count_feature_act(act):
    
    # 获取每个author对应的video sum
    temp = act.groupby(['user_id','author_id'])['activity_count'].sum()
    temp = temp.reset_index()
    
    # 将结果转换成list：一个是author名单，一个是对应的video sum值
    author_list = temp.groupby('user_id')['author_id'].agg(lambda x: list(x)).reset_index()
    author_list.columns = ['user_id','author_list']
    author_video_count_list = temp.groupby('user_id')['activity_count'].agg(lambda x: list(x)).reset_index()
    author_video_count_list.columns = ['user_id','author_video_count_list']
    
    # 合并结果
    author_count = pd.merge(author_list,author_video_count_list,how = 'outer')
    
    return author_count

In [ ]:
step1 给出聚集的list

In [62]:
act_author_count = author_id_count_feature_act(act)

In [63]:
act_author_count.head()

,user_id,author_list,author_video_count_list
0,8,"[12378, 37639, 57988, 91494, 93934, 101802, 10...","[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,73,"[287045, 299465, 356372, 400197, 422123, 45799...","[10, 1, 1, 1, 2, 1, 1, 1, 7, 1, 1, 1, 1, 1, 4]"
2,129,[918207],[1]
3,228,[529655],[22]
4,351,"[34289, 40579, 71808, 74555, 88680, 105164, 14...","[1, 1, 1, 1, 4, 1, 1, 4, 1, 3, 1, 9, 5, 3, 6, ..."


In [ ]:
step2 统计用户的行为热度
list：将每个author的对应的video观看量作为list，
统计：
author的数量，video的总量 / author的数量：
这两个实际可以看做author_video_count_list的len，average，sum（前边count统计过了）
average的意义——用户偏好：author数量少，video数量又大的。video / count的值

其余的统计量：
max，max占sum的比例值
std：衡量波动情况，看的作者是否杂，杂的user没有关注的用户，容易弃坑
峰度和偏度：刻画list的变化趋势（需要先给list排序）

In [64]:
def author_count_feature_act(act_author_count,time_gap_right):
    
    author_len = [] # author的数量
    author_average = [] # 用户偏好均值
    author_max = [] # 最偏好的max
    author_max_ratio = [] # 占sum的比例
    author_std = [] # mean附近的稳定性，如果count悬殊非常大则会波动厉害，说明有区分度
    # 需要先排序，统一变化趋势
    author_skew = [] # 偏度
    author_kurtosis = [] # 峰度
    for i in range(act_author_count.shape[0]):
    # for i in range(10):
        item = act_author_count.iloc[i,2] 
        
        author_len.append(len(item))
        author_average.append(np.mean(item))
        author_max.append(np.max(item))
        author_max_ratio.append(np.max(item) / np.sum(item))
        author_std.append(np.std(item))
        temp = sorted(item,reverse = True)
        author_skew.append(stats.skew(temp))
        author_kurtosis.append(stats.kurtosis(temp))
    
    act_author_count['act_author_count_len'] = author_len
    act_author_count['act_author_count_average'] = author_average
    act_author_count['act_author_count_max'] = author_max
    act_author_count['act_author_count_max_ratio'] = author_max_ratio
    act_author_count['act_author_count_std'] = author_std
    act_author_count['act_author_count_skew'] = author_skew
    act_author_count['act_author_count_kurtosis'] = author_kurtosis
      
    return act_author_count

In [65]:
time_gap_right = 16 # 区间的最右端
act_author_count = author_count_feature_act(act_author_count,time_gap_right)

In [66]:
act_author_count.head()

,user_id,author_list,author_video_count_list,act_author_count_len,act_author_count_average,act_author_count_max,act_author_count_max_ratio,act_author_count_std,act_author_count_skew,act_author_count_kurtosis
0,8,"[12378, 37639, 57988, 91494, 93934, 101802, 10...","[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",60,1.600000,7,0.072917,1.280625,2.586390,6.403331
1,73,"[287045, 299465, 356372, 400197, 422123, 45799...","[10, 1, 1, 1, 2, 1, 1, 1, 7, 1, 1, 1, 1, 1, 4]",15,2.266667,10,0.294118,2.619584,2.044814,2.826957
2,129,[918207],[1],1,1.000000,1,1.000000,0.000000,0.000000,-3.000000
3,228,[529655],[22],1,22.000000,22,1.000000,0.000000,0.000000,-3.000000
4,351,"[34289, 40579, 71808, 74555, 88680, 105164, 14...","[1, 1, 1, 1, 4, 1, 1, 4, 1, 3, 1, 9, 5, 3, 6, ...",47,3.744681,31,0.176136,5.777683,3.471943,12.324511


In [ ]:
（3）将行为table的所有特征整合到一起

In [67]:
act_day_and_count.shape,act_page_and_type.shape,act_author_count.shape

((20535, 40), (20535, 12), (20535, 10))

In [68]:
temp1 = pd.merge(act_day_and_count,act_page_and_type,how = 'outer')

In [69]:
temp1.shape

(20535, 51)

In [70]:
temp2 = pd.merge(temp1,act_author_count,how = 'outer')

In [71]:
temp2.shape

(20535, 60)

In [72]:
temp2.head()

,user_id,activity_day,act_day_gap_list,activity_count,act_count_gap_list,act_day_near_pred,act_day_sum,act_day_sum_average,act_day_gap_max,act_day_gap_ain,...,action_type_5,author_list,author_video_count_list,act_author_count_len,act_author_count_average,act_author_count_max,act_author_count_max_ratio,act_author_count_std,act_author_count_skew,act_author_count_kurtosis
0,8,"[9, 10, 11, 13]","[1, 1, 2]","[21, 39, 5, 31]","[18, -34, 26]",3,4,0.8,2,1,...,0.0,"[12378, 37639, 57988, 91494, 93934, 101802, 10...","[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",60,1.600000,7,0.072917,1.280625,2.586390,6.403331
1,73,"[4, 5]",[1],"[5, 29]",[24],11,2,1.0,1,1,...,0.0,"[287045, 299465, 356372, 400197, 422123, 45799...","[10, 1, 1, 1, 2, 1, 1, 1, 7, 1, 1, 1, 1, 1, 4]",15,2.266667,10,0.294118,2.619584,2.044814,2.826957
2,129,[12],[-1],[1],NaN,4,1,1.0,-1,-1,...,0.0,[918207],[1],1,1.000000,1,1.000000,0.000000,0.000000,-3.000000
3,228,[13],[-1],[22],NaN,3,1,1.0,-1,-1,...,0.0,[529655],[22],1,22.000000,22,1.000000,0.000000,0.000000,-3.000000
4,351,[3],[-1],[176],NaN,13,1,1.0,-1,-1,...,0.0,"[34289, 40579, 71808, 74555, 88680, 105164, 14...","[1, 1, 1, 1, 4, 1, 1, 4, 1, 3, 1, 9, 5, 3, 6, ...",47,3.744681,31,0.176136,5.777683,3.471943,12.324511


In [ ]:
存档

In [73]:
temp2.to_csv(
    'dataset_B_for_model_method/temp_dataset/act_1_16_table_action_feature.csv',index = False)

##### 获得最终的特征集

In [ ]:
将所有的特征集合并到一起
（3个table在这里可能有些不同）

In [74]:
table123_feature = pd.read_csv(
    'dataset_B_for_model_method/temp_dataset/table123_1_16_first_3_feature.csv')
act_feature = pd.read_csv(
    'dataset_B_for_model_method/temp_dataset/act_1_16_table_action_feature.csv')

In [75]:
table123_feature.shape,act_feature.shape

((23923, 44), (20535, 60))

In [76]:
table123_feature.isnull().sum().sum()

0

In [77]:
table_all = pd.merge(table123_feature,act_feature,how = 'outer')

In [78]:
table_all.shape

(23923, 103)

In [79]:
table_all.head()

,user_id,app_launch_day,launch_gap_list,launch_day_near_pred,launch_day_sum,launch_day_sum_average,launch_day_gap_max,launch_day_gap_ain,launch_day_gap_average,launch_day_gap_std,...,action_type_5,author_list,author_video_count_list,act_author_count_len,act_author_count_average,act_author_count_max,act_author_count_max_ratio,act_author_count_std,act_author_count_skew,act_author_count_kurtosis
0,8,"[9.0, 10.0, 11.0, 13.0]","[1.0, 1.0, 2.0]",3.0,4,0.80,2.0,1.0,1.333333,0.471405,...,0.0,"[12378, 37639, 57988, 91494, 93934, 101802, 10...","[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",60.0,1.600000,7.0,0.072917,1.280625,2.586390,6.403331
1,73,"[2.0, 4.0, 5.0]","[2.0, 1.0]",11.0,3,0.75,2.0,1.0,1.500000,0.500000,...,0.0,"[287045, 299465, 356372, 400197, 422123, 45799...","[10, 1, 1, 1, 2, 1, 1, 1, 7, 1, 1, 1, 1, 1, 4]",15.0,2.266667,10.0,0.294118,2.619584,2.044814,2.826957
2,129,[12.0],[-1],4.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,...,0.0,[918207],[1],1.0,1.000000,1.0,1.000000,0.000000,0.000000,-3.000000
3,228,[13.0],[-1],3.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,...,0.0,[529655],[22],1.0,22.000000,22.0,1.000000,0.000000,0.000000,-3.000000
4,351,[3.0],[-1],13.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,...,0.0,"[34289, 40579, 71808, 74555, 88680, 105164, 14...","[1, 1, 1, 1, 4, 1, 1, 4, 1, 3, 1, 9, 5, 3, 6, ...",47.0,3.744681,31.0,0.176136,5.777683,3.471943,12.324511


In [ ]:
对待null的态度
——比较方便的态度是，将缺少act的user和有act但是某些特征没法取值的情况，看成两类。前者是nan，后者是缺省值-1或者nan

In [ ]:
添加y

In [80]:
y['y'] = 1

In [81]:
table_all = pd.merge(table_all,y,how = 'outer')
table_all['y'] = table_all['y'].fillna(0)
table_all['y'].value_counts()

1.0    12430
0.0    11493
Name: y, dtype: int64

In [82]:
table_all.head()

,user_id,app_launch_day,launch_gap_list,launch_day_near_pred,launch_day_sum,launch_day_sum_average,launch_day_gap_max,launch_day_gap_ain,launch_day_gap_average,launch_day_gap_std,...,author_list,author_video_count_list,act_author_count_len,act_author_count_average,act_author_count_max,act_author_count_max_ratio,act_author_count_std,act_author_count_skew,act_author_count_kurtosis,y
0,8,"[9.0, 10.0, 11.0, 13.0]","[1.0, 1.0, 2.0]",3.0,4,0.80,2.0,1.0,1.333333,0.471405,...,"[12378, 37639, 57988, 91494, 93934, 101802, 10...","[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",60.0,1.600000,7.0,0.072917,1.280625,2.586390,6.403331,1.0
1,73,"[2.0, 4.0, 5.0]","[2.0, 1.0]",11.0,3,0.75,2.0,1.0,1.500000,0.500000,...,"[287045, 299465, 356372, 400197, 422123, 45799...","[10, 1, 1, 1, 2, 1, 1, 1, 7, 1, 1, 1, 1, 1, 4]",15.0,2.266667,10.0,0.294118,2.619584,2.044814,2.826957,0.0
2,129,[12.0],[-1],4.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,...,[918207],[1],1.0,1.000000,1.0,1.000000,0.000000,0.000000,-3.000000,1.0
3,228,[13.0],[-1],3.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,...,[529655],[22],1.0,22.000000,22.0,1.000000,0.000000,0.000000,-3.000000,1.0
4,351,[3.0],[-1],13.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,...,"[34289, 40579, 71808, 74555, 88680, 105164, 14...","[1, 1, 1, 1, 4, 1, 1, 4, 1, 3, 1, 9, 5, 3, 6, ...",47.0,3.744681,31.0,0.176136,5.777683,3.471943,12.324511,0.0


In [ ]:
观察table是否包含了登录时间在1-16号的全部用户

In [83]:
user_register_log = pd.read_csv('dataset_B/user_register_log.csv')
user_register_log = user_register_log[user_register_log['register_day'] < 17]
register_user = user_register_log['user_id']
table_all_user = table_all['user_id']
other_user = list(set(register_user) - set(table_all_user))

In [84]:
len(other_user),len(register_user),len(table_all_user)

(0, 23923, 23923)

In [ ]:
如果不是空集，需要把这部分数据并入最终的数据集中
（注意y的填补用0，其余用-1）

In [ ]:
other_user = pd.DataFrame(other_user)
other_user.columns = ['user_id']

In [ ]:
存档最终数据集

In [85]:
table_all.to_csv(
    'dataset_B_for_model_method/final_dataset/complete_dataset_1_16.csv',index = False)